In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
import os

print("--- Part 2: Creating the Initial Model ---")

# --- Configuration with your specific paths ---
INPUT_DIR = r"D:\FedShield,Personal\wataiData\csv\CICIoT2023\federated_data"
SERVER_DF_PATH = os.path.join(INPUT_DIR, 'server_df.csv')

# --- Load Server Data ---
print(f"Loading server data from {SERVER_DF_PATH}...")
server_df = pd.read_csv(SERVER_DF_PATH)

# --- Preprocessing ---
print("Preprocessing data...")
X_server = server_df.drop('label', axis=1)
y_server = server_df['label']

label_encoder = LabelEncoder()
y_server_encoded = label_encoder.fit_transform(y_server)

scaler = StandardScaler()
X_server_scaled = scaler.fit_transform(X_server)

with open(os.path.join(INPUT_DIR, 'label_encoder.pkl'), 'wb') as f:
    pickle.dump(label_encoder, f)
with open(os.path.join(INPUT_DIR, 'scaler.pkl'), 'wb') as f:
    pickle.dump(scaler, f)
print("Saved label_encoder.pkl and scaler.pkl.")

# --- Build and Train Model ---
print("Building and training the initial model...")

def create_model(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(input_shape,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

initial_model = create_model(X_server_scaled.shape[1], len(label_encoder.classes_))
initial_model.fit(X_server_scaled, y_server_encoded, epochs=5, batch_size=256, verbose=1)

# --- Extract and Save Weights ---
print("Extracting and saving model weights...")
initial_weights = initial_model.get_weights()
with open(os.path.join(INPUT_DIR, 'initial_weights.pkl'), 'wb') as f:
    pickle.dump(initial_weights, f)

print(f"\n✅ Part 2 Complete!")
print(f"Initial model weights saved to: {os.path.join(INPUT_DIR, 'initial_weights.pkl')}")